<a href="https://colab.research.google.com/github/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week5/LinearModels_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing the objective functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('https://github.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/blob/main/Week5/Week5_statistics_exercise.csv?raw=true')
dictionary = pd.read_csv('https://raw.githubusercontent.com/araldi/HS21---Big-Data-Analysis-in-Biomedical-Research-376-1723-00L-/main/Week3/Week3_homework_dictionary_part1.csv')

Let's check the relationship between IGF-1 and age.

In [ ]:
df.head()

In [ ]:
dictionary

In [ ]:
df.dropna(subset = ['21022-0.0', '30770-0.0'], inplace =True)

In [ ]:
# Let's look at the data
plt.figure(figsize = (10,10))
plt.scatter(df['21022-0.0'], df['30770-0.0'], s=1)
plt.ylim(0, 60)

In [ ]:
from scipy.optimize import minimize
def line(x, b, a): #function to minimize
    return a * x + b #line equation

def fitfunc(args, x, y):
    a, b = args
    return sum((y - line(a, b, x))**2) # L2 objective function
 
x = df['21022-0.0']
y = df['30770-0.0']
initialGuess = (0, 40)
 
fitfunc(initialGuess, x, y)
solution = minimize(fitfunc, initialGuess, args=(x, y)) # minimize the objective function

In [ ]:
solution

In [ ]:
a, b = solution['x'][0], solution['x'][1]

In [ ]:
plt.figure(figsize = (10,10))
plt.scatter(df['21022-0.0'], df['30770-0.0'], s=2, c='brown', alpha = 0.1)
plt.ylim(0, 60)

x = np.linspace(df['21022-0.0'].min(), df['21022-0.0'].max())
y = a * x + b

plt.plot(x, y, color = 'k', linestyle = '--')
plt.xlabel('Age [Years]')
plt.ylabel('IGF-1 [nmol/L]')
plt.title('Linear relationship between IGF-1 and age')

In [ ]:
def line(a, b, c, x): #function to minimize
    return a * x**2 + b * x + c #second order equation

def fitfunc(args, x, y):
    a, b, c = args
    return sum((y - line(a, b, c, x))**2) # L2 objective function
                                          # sum of squared residuals
 
x = df['21022-0.0']
y = df['30770-0.0']
initialGuess = (1, 1, 1)
 
fitfunc(initialGuess, x, y)
solution = minimize(fitfunc, initialGuess, args=(x, y)) # minimize the objective function

In [ ]:
solution

In [ ]:
a, b, c = solution['x'][0], solution['x'][1], solution['x'][2]

x = df['21022-0.0']
y = df['30770-0.0']

plt.figure(figsize = (10,10))
plt.scatter(df['21022-0.0'], df['30770-0.0'], s=2, c='brown', alpha = 0.1)
plt.ylim(0, 60)

x = np.linspace(df['21022-0.0'].min(), df['21022-0.0'].max())
y = a * x**2 + b * x + c

plt.plot(x, y, color = 'k', linestyle = '--')
plt.xlabel('Age [Years]')
plt.ylabel('IGF-1 [nmol/L]')
plt.title('Second order relationship between IGF-1 and age')

### Use np.polyfit

In [ ]:
# first order with polyfit
x = df['21022-0.0']
y = df['30770-0.0']
args = np.polyfit(x,y,1)

In [ ]:
a, b = args
plt.figure(figsize = (10,10))
plt.scatter(df['21022-0.0'], df['30770-0.0'], s=2, c='brown', alpha = 0.1)
plt.ylim(0, 60)

x = np.linspace(df['21022-0.0'].min(), df['21022-0.0'].max())
y = a * x + b

plt.plot(x, y, color = 'k', linestyle = '--')
plt.xlabel('Age [Years]')
plt.ylabel('IGF-1 [nmol/L]')
plt.title('Linear relationship between IGF-1 and age')

In [ ]:
# second order with polyfit
x = df['21022-0.0']
y = df['30770-0.0']
args = np.polyfit(x,y,2)

In [ ]:
a, b, c = args

plt.figure(figsize = (10,10))
plt.scatter(df['21022-0.0'], df['30770-0.0'], s=2, c='brown', alpha = 0.1)
plt.ylim(0, 60)

x = np.linspace(df['21022-0.0'].min(), df['21022-0.0'].max())
y = a * x**2 + b * x + c

plt.plot(x, y, color = 'k', linestyle = '--')
plt.xlabel('Age [Years]')
plt.ylabel('IGF-1 [nmol/L]')
plt.title('Second order relationship between IGF-1 and age')

# Multiple linear regression

In [ ]:
dictionary['Description_cols'] = dictionary['Description'].str.split(' ').str[0]

In [ ]:
dict_columns = {}
for index, value in enumerate(dictionary['Code']):
  dict_columns[value] = dictionary.loc[index, "Description_cols"]

In [ ]:
df.rename(columns = dict_columns, inplace = True)

In [ ]:
dictionary

In [ ]:
# prepare the data
df = df.dropna()

### Test the assumptions

In [ ]:
# Check the residuals for glucose
# according to assumptions, they have to be normally distributed

#define figure size
fig = plt.figure(figsize=(12,8))

#produce regression plots
fig = sm.graphics.plot_regress_exog(res, 'Glucose', fig=fig)

In [ ]:
# check covariance

sns.pairplot(df.select_dtypes(exclude = 'int64'))
# took 2 minutes to run

In [ ]:
# Plot heatmap for feature covariance
plt.figure(figsize = (14,10))
sns.set(font_scale = 2)
s = sns.heatmap(df.select_dtypes(exclude = 'int64').cov())

In [ ]:
# importing module
import statsmodels.api as sm
import statsmodels.formula.api as smf

# fitting the data
mod = smf.ols(formula='Glycated ~ Glucose', data=df)

res = mod.fit()
res.summary()

In [ ]:
# standardize the data
def standardize(x):
  return (x - np.mean(x))/np.std(x)

In [ ]:
for i in df.select_dtypes(exclude = 'int64'):
  df[i] = standardize(df[i])


In [ ]:
df.select_dtypes(exclude = 'int64'). describe()

In [ ]:
# importing module
import statsmodels.api as sm
import statsmodels.formula.api as smf

# fitting the data
mod = smf.ols(formula='Glycated ~ Glucose', data=df)

res = mod.fit()
res.summary()

### Exercise
Play around with dependent variables to get the best possible model that describes (and predicts) glucose and LDL

In [ ]:
# example
mod = smf.ols(formula='Glycated ~ Glucose + Q("IGF-1") + AgeRecruit', data=df)
res = mod.fit()
res.summary()